Imports de codigo

In [37]:
import requests
import pandas as pd
import re
import sys
import os
from tqdm import tqdm
import random
import time
src_path = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path.append(src_path)
from Funciones import crear_lista_fechas
from Funciones import convertir_duracion_a_minutos
from Bibliotecarecursos import ciudades_aeropuertos
import os
import dotenv as dotenv

Datos a introducir

In [38]:
dotenv.load_dotenv()
api_key=os.getenv("apikey_amadeus")
api_secret=os.getenv("apisecret_amadeus")

# Fecha de búsqueda
fecha_inicio = "2025-02-1"  # Cambia esta fecha por la que necesites en formato YYYY-MM-DD
fecha_fin= "2025-02-28"
listadias=crear_lista_fechas(fecha_inicio,fecha_fin)

# Ciudades de salida y destino
Ciudad_salida = "Madrid"
Ciudad_destino = "Roma"

def obtener_aeropuertos(ciudad):
    ciudad_formateada = ciudad.title()  
    if ciudad_formateada in ciudades_aeropuertos:
        return ciudades_aeropuertos[ciudad_formateada]
    else:
        return f"No se encontraron aeropuertos para {ciudad}."
aeropuertos_salida=obtener_aeropuertos(Ciudad_salida)
aeropuertos_destino=obtener_aeropuertos(Ciudad_destino)

Codigo de extraccion de API Amadeus for Developers

[Documentación API Amadeus](https://developers.amadeus.com/?s_kwcid=AL!10969!3!452455070626!b!!g!!%2Bamadeus%20%2Btravel%20%2Bapis&gad_source=1&gclid=Cj0KCQjwsc24BhDPARIsAFXqAB17PIkmYaVnDyaOL0svq05tBzjJv4IF9_zhkEskFYY89G8GTKm7orsaAsq7EALw_wcB)


In [39]:
# Función para obtener el token de acceso
def obtener_token_acceso(api_key, api_secret):
    urltoken = "https://test.api.amadeus.com/v1/security/oauth2/token"
    encabezados = {"Content-Type": "application/x-www-form-urlencoded"}
    datos = {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }
    
    respuesta = requests.post(urltoken, headers=encabezados, data=datos)
    
    if respuesta.status_code == 200:
        return respuesta.json().get('access_token')
    else:
        raise Exception(f"Error en la autenticación: {respuesta.status_code} - {respuesta.text}")

# Función para buscar vuelos con precios, solo vuelos directos
def obtener_datos(token_acceso, origen, destino, fecha_salida, intentos=0):
    url = f"https://test.api.amadeus.com/v2/shopping/flight-offers"
    encabezados = {
        "Authorization": f"Bearer {token_acceso}"
    }
    
    parametros = {
        "originLocationCode": origen,              # Código IATA del origen 
        "destinationLocationCode": destino,        # Código IATA del destino (solo un destino a la vez)
        "departureDate": fecha_salida,             # Fecha de salida en formato YYYY-MM-DD
        "adults": 1,                               # Número de adultos
        "travelClass": "ECONOMY",                  # Clase de viaje
        "currencyCode": "EUR"                     # Moneda en euros
    }
    
    respuesta = requests.get(url, headers=encabezados, params=parametros)
    
    if respuesta.status_code == 200:
        return respuesta.json().get('data', [])
    
    elif respuesta.status_code == 429:
        # Exponential backoff: espera cada vez más tiempo
        wait_time = 2 ** intentos + random.uniform(0, 1)  # Espera exponencial con aleatoriedad
        print(f"Rate limit excedido, esperando {wait_time:.2f} segundos antes de reintentar...")
        time.sleep(wait_time)
        return obtener_datos(token_acceso, aeropuerto1, aeropuerto2, dia, intentos + 1)

    else:
        print(f"Error al obtener vuelos: {respuesta.status_code} - {respuesta.text}")
        return []

# Obtener el token de acceso
token_acceso = obtener_token_acceso(api_key, api_secret)

# # Lista para almacenar los datos de vuelos de todos los destinos
datos_vuelos_ida = []

# Buscar dia del mes
for dia in tqdm(listadias):
# Buscar vuelos para cada aeropuerto de salida
    for aeropuerto1 in aeropuertos_salida:
    # Buscar vuelos para cada aeropuerto de destino
        for aeropuerto2 in aeropuertos_destino:
            lista_vuelos = obtener_datos(token_acceso, aeropuerto1, aeropuerto2, dia)
            # Recopilar los datos relevantes de cada vuelo, solo si tiene 1 segmento (vuelo directo)
            for vuelo in lista_vuelos:
                info_precio = vuelo.get('price', {})
                precio_total = info_precio.get('total', 'N/A')  # Precio total del vuelo
                for itinerario in vuelo.get('itineraries', []):
                    # Asegurarse de que el itinerario tiene solo 1 segmento (vuelo directo)
                    if len(itinerario.get('segments', [])) == 1:
                        for segmento in itinerario.get('segments', []):
                            info_vuelo = {
                                "Fecha de Salida Completa": segmento["departure"]["at"],
                                "Aeropuerto de Salida": segmento["departure"]["iataCode"],
                                "Aeropuerto de Llegada": segmento["arrival"]["iataCode"],
                                "Código de la Aerolínea": segmento["carrierCode"],
                                "Número de Vuelo": segmento["number"],
                                "Duración (Minutos)": convertir_duracion_a_minutos(itinerario["duration"]),  # Duración en minutos
                                "Precio (EUR)": precio_total  # Agregar el precio total del vuelo
                            }
                            datos_vuelos_ida.append(info_vuelo)

# Lista para almacenar los datos de vuelos de todos los destinos
datos_vuelos_vuelta = []

# Buscar dia del mes
for dia in tqdm(listadias):
# Buscar vuelos para cada aeropuerto de salida
    for aeropuerto1 in aeropuertos_salida:
    # Buscar vuelos para cada aeropuerto de destino
        for aeropuerto2 in aeropuertos_destino:
            lista_vuelos = obtener_datos(token_acceso, aeropuerto2, aeropuerto1, dia)
            # Recopilar los datos relevantes de cada vuelo, solo si tiene 1 segmento (vuelo directo)
            for vuelo in lista_vuelos:
                info_precio = vuelo.get('price', {})
                precio_total = info_precio.get('total', 'N/A')  # Precio total del vuelo
                for itinerario in vuelo.get('itineraries', []):
                    # Asegurarse de que el itinerario tiene solo 1 segmento (vuelo directo)
                    if len(itinerario.get('segments', [])) == 1:
                        for segmento in itinerario.get('segments', []):
                            info_vuelo = {
                                "Fecha de Salida Completa": segmento["departure"]["at"],
                                "Aeropuerto de Salida": segmento["departure"]["iataCode"],
                                "Aeropuerto de Llegada": segmento["arrival"]["iataCode"],
                                "Código de la Aerolínea": segmento["carrierCode"],
                                "Número de Vuelo": segmento["number"],
                                "Duración (Minutos)": convertir_duracion_a_minutos(itinerario["duration"]),  # Duración en minutos
                                "Precio (EUR)": precio_total  # Agregar el precio total del vuelo
                            }
                            datos_vuelos_vuelta.append(info_vuelo)

# Crear un DataFrame con los datos de los vuelos
df_vuelos_ida = pd.DataFrame(datos_vuelos_ida)

# Convertir las columnas de fechas y horas a tipo datetime
df_vuelos_ida['Fecha de Salida Completa'] = pd.to_datetime(df_vuelos_ida['Fecha de Salida Completa'])

# Separar la fecha y la hora en dos columnas diferentes
df_vuelos_ida['Fecha de Salida'] = df_vuelos_ida['Fecha de Salida Completa'].dt.date
df_vuelos_ida['Hora de Salida'] = df_vuelos_ida['Fecha de Salida Completa'].dt.time

# Eliminar las columnas originales si ya no son necesarias
df_vuelos_ida.drop(columns=['Fecha de Salida Completa'], inplace=True)

# Crear columna de fecha de extracción
df_vuelos_ida['Fecha de extracción']= pd.Timestamp.today().date()

# Crear un DataFrame con los datos de los vuelos
df_vuelos_vuelta = pd.DataFrame(datos_vuelos_vuelta)

# Convertir las columnas de fechas y horas a tipo datetime
df_vuelos_vuelta['Fecha de Salida Completa'] = pd.to_datetime(df_vuelos_vuelta['Fecha de Salida Completa'])

# Separar la fecha y la hora en dos columnas diferentes
df_vuelos_vuelta['Fecha de Salida'] = df_vuelos_vuelta['Fecha de Salida Completa'].dt.date
df_vuelos_vuelta['Hora de Salida'] = df_vuelos_vuelta['Fecha de Salida Completa'].dt.time

# Eliminar las columnas originales si ya no son necesarias
df_vuelos_vuelta.drop(columns=['Fecha de Salida Completa'], inplace=True)

# Crear columna de fecha de extracción
df_vuelos_vuelta['Fecha de extracción']= pd.Timestamp.today().date()



 79%|███████▊  | 22/28 [00:58<00:16,  2.81s/it]

Rate limit excedido, esperando 1.60 segundos antes de reintentar...


 50%|█████     | 14/28 [00:38<00:36,  2.62s/it]

Rate limit excedido, esperando 1.81 segundos antes de reintentar...


 61%|██████    | 17/28 [00:48<00:33,  3.00s/it]

Rate limit excedido, esperando 1.47 segundos antes de reintentar...


100%|██████████| 28/28 [01:21<00:00,  2.91s/it]


In [40]:
# Exportación del DataFrame a un archivo CSV
df_vuelos_ida.to_csv(f'../data/vuelos_{Ciudad_salida}_{Ciudad_destino}_de_{fecha_inicio}_a_{fecha_fin}_directos.csv', index=False)
df_vuelos_vuelta.to_csv(f'../data/vuelos_{Ciudad_destino}_{Ciudad_salida}_de_{fecha_inicio}_a_{fecha_fin}_directos.csv', index=False)
